## Imoprt LIbrary



In [ ]:
pip install -q tensorflow-recommenders

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.2/96.2 kB 4.7 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import tensorflow_recommenders as tfrs
from google.colab import drive


## Import data from CSV

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
makanan = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/model/Dataset_Nutrisi_Makanan.csv")
users = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/model/data_user.csv")

##Preprocessing Data

In [ ]:
makanan= tf.data.Dataset.from_tensor_slices(dict(makanan))

In [ ]:
users= tf.data.Dataset.from_tensor_slices(dict(users))

In [ ]:
users = users.map(lambda x: {
    "Nama_Makanan": x["Nama_Makanan"],
    "user_id": x["user_id"],
})
makanan = makanan.map(lambda x: x["Nama_Makanan"])

In [ ]:
tf.random.set_seed(42)
shuffled = users.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(80_000)
test = shuffled.skip(80_000).take(20_000)

In [ ]:
All_Nama_Makanan = makanan.batch(1_000)
user_ids = users.batch(1_000_000).map(lambda x: x["user_id"])

Nama_Makanan_Unik= np.unique(np.concatenate(list(All_Nama_Makanan)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

Nama_Makanan_Unik[:10]

array([b'Ayam Panggang (1 Porsi - 85 g)', b'Bakso Ayam (1 Porsi)',
       b'Bakso Sapi (100 g)', b'Beef Patty (1 Porsi)',
       b'Bola Ikan Udang Wortel (1 Porsi)', b'Buah Pir (100 g)',
       b'Bubur Jagung (1 Porsi)', b'Bubur Kacang Hijau (1 Porsi)',
       b'Buncis (100 g)', b'Cauliflower Fried Rice (1 Porsi)'],
      dtype=object)

In [ ]:
unique_user_ids[:10]

array([b'user-00123456', b'user-00234567', b'user-00345678',
       b'user-00456789', b'user-00567900', b'user-00679011',
       b'user-00790122', b'user-00901233', b'user-01012344',
       b'user-01123455'], dtype=object)

##Modelling

In [ ]:
user_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(vocabulary=unique_user_ids, mask_token=None),
  tf.keras.layers.Embedding(len(unique_user_ids) + 1, 32)
])

In [ ]:
makanan_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(vocabulary=Nama_Makanan_Unik, mask_token=None),
  tf.keras.layers.Embedding(len(Nama_Makanan_Unik) + 1, 32)
])

In [ ]:
task = tfrs.tasks.Retrieval(
  metrics= tfrs.metrics.FactorizedTopK(candidates=makanan.batch(128).map(makanan_model))
)

#Recommended system

In [ ]:
from typing import Dict, Text
class MakananlensModel(tfrs.Model):

  def __init__(self, user_model, makanan_model):
    super().__init__()
    self.makanan_model: tf.keras.Model = makanan_model
    self.user_model: tf.keras.Model = user_model
    self.task: tf.keras.layers.Layer = task

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    # We pick out the user features and pass them into the user model.
    user_embeddings = self.user_model(features["user_id"])
    # And pick out the makanan features and pass them into the makanan model,
    # getting embeddings back.
    makanan_embeddings = self.makanan_model(features["Nama_Makanan"])

    # The task computes the loss and the metrics.
    return self.task(user_embeddings, makanan_embeddings)


In [ ]:
def get_model():
  return MakananlensModel(user_model, makanan_model)

In [ ]:
model =MakananlensModel(user_model, makanan_model)
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

In [ ]:
cached_train = train.shuffle(100_000).batch(8192).cache()
cached_test = test.batch(100).cache()

In [ ]:
model.fit(cached_train, epochs=3)

Epoch 1/3
1/1 [==============================] - 1s 1s/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 1.0000 - factorized_top_k/top_10_categorical_accuracy: 1.0000 - factorized_top_k/top_50_categorical_accuracy: 1.0000 - factorized_top_k/top_100_categorical_accuracy: 1.0000 - loss: 66.6248 - regularization_loss: 0.0000e+00 - total_loss: 66.6248
Epoch 2/3
1/1 [==============================] - 0s 92ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 1.0000 - factorized_top_k/top_10_categorical_accuracy: 1.0000 - factorized_top_k/top_50_categorical_accuracy: 1.0000 - factorized_top_k/top_100_categorical_accuracy: 1.0000 - loss: 65.6151 - regularization_loss: 0.0000e+00 - total_loss: 65.6151
Epoch 3/3
1/1 [==============================] - 0s 92ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 1.0000 - fa

In [ ]:
#model.evaluate(cached_test, return_dict=True)

In [ ]:
# Create a model that takes in raw query features, and
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
index.index_from_dataset(
  tf.data.Dataset.zip((makanan.batch(100), makanan.batch(100).map(model.makanan_model)))
)

# Get recommendations.
_, food = index(tf.constant(["10"]))
print(f"Recommendations for user 20: {food[0, :5]}")

Recommendations for user 20: [b'Pisang (1 Potong)' b'Bubur Kacang Hijau (1 Porsi)'
 b'Sup Telur Pedas (1 Porsi)' b'Nasi Goreng Oatmeal (1 Porsi)'
 b'Ikan Panggang (1 Porsi)']


In [ ]:
import os

tmp  = r'/content/drive/MyDrive/Colab Notebooks/model'
path = os.path.join(tmp, "model")

  # Save the index.
tf.saved_model.save(index, path)



In [ ]:
mode = "Speed"

if mode == 'Storage':
    optimization = tf.lite.Optimize.OPTIMIZE_FOR_SIZE
elif mode == 'Speed':
    optimization = tf.lite.Optimize.OPTIMIZE_FOR_LATENCY
else:
    optimization = tf.lite.Optimize.DEFAULT

In [ ]:
converter = tf.lite.TFLiteConverter.from_saved_model(path)

# Set the optimzations
converter.optimizations = [optimization]

# Invoke the converter to finally generate the TFLite model
tflite_model = converter.convert()

In [ ]:
import pathlib
tflite_model_file = pathlib.Path('/content/drive/MyDrive/Colab Notebooks/model/model.tflite')
tflite_model_file.write_bytes(tflite_model)

10616

In [ ]:
  # Load it back; can also be done in TensorFlow Serving.
loaded = tf.saved_model.load(path)

  # Pass a user id in, get top predicted movie titles back.
scores, titles = loaded(["42"])

print(f"Recommendations: {titles[0][:3]}")

Recommendations: [b'Pisang (1 Potong)' b'Nasi Goreng Oatmeal (1 Porsi)'
 b'Bubur Kacang Hijau (1 Porsi)']
